In [2]:
import pandas as pd
import numpy as np
from pydicom import dcmread
import sys

sys.path.insert(0, '../../data/SpineNet')
import spinenet
from spinenet import SpineNet, download_example_scan
from spinenet.io import load_dicoms_from_folder

spnt = SpineNet(device='cuda:0', verbose=True)

C:\Users\Victor\.conda\envs\python-doodles\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Victor\.conda\envs\python-doodles\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loading Detection Model...
==> Loading model trained for 436 epochs...
Loading Appearance Model...
==> Loading model trained for 188 epochs...
Loading Context Model...
==> Loading model trained for 17 epochs...
Loading Grading Model...
==> Loading model trained for 2 epochs...


In [3]:
LEVELS = ["L1", "L2", "L3", "L4", "L5", "S1"]
COLORS = {
    "L1": "red",
    "L2": "blue",
    "L3": "green",
    "L4": "yellow",
    "L5": "white",
    "S1": "purple"
}

In [6]:
train_descs_path = "../../data/rsna-2024-lumbar-spine-degenerative-classification/train_series_descriptions.csv"
train_images_path = "../../data/rsna-2024-lumbar-spine-degenerative-classification/train_images"

In [7]:
train_descs = pd.read_csv(train_descs_path)
train_descs = train_descs[train_descs["series_description"] == "Sagittal T1"]
train_descs

,study_id,series_id,series_description
1,4003253,1054713880,Sagittal T1
4,4646740,3486248476,Sagittal T1
8,7143189,3219733239,Sagittal T1
10,8785691,1570286759,Sagittal T1
14,10728036,2399638375,Sagittal T1
...,...,...,...
6281,4282019580,3029774733,Sagittal T1
6283,4283570761,2708429184,Sagittal T1
6285,4284048608,1875151370,Sagittal T1
6288,4287160193,327893304,Sagittal T1


In [8]:
def calculate_centers(data):
    centers = {}
    for item in data:
        level = item["predicted_label"]
        if level in LEVELS:
            average_polygon = item["average_polygon"]
            centroid_x = np.mean(average_polygon[:, 0])
            centroid_y = np.mean(average_polygon[:, 1])
            centroid_z = item["slice_nos"][len(item["slice_nos"])//2]
            centers[level] = (centroid_x, centroid_y, centroid_z)
    return centers

In [9]:
centers_per_study = {
    "study_id": [],
    "series_id": [],
    "x": [],
    "y": [],
    "instance_number": [],
    "level": []
}


In [10]:
from tqdm import tqdm

for index, row in tqdm(train_descs.iterrows()):
    scan = load_dicoms_from_folder(f"{train_images_path}/{row['study_id']}/{row['series_id']}", require_extensions=False)
    num_slices = scan.volume.shape[-1]

    vert_dicts = spnt.detect_vb(scan.volume, scan.pixel_spacing)
    centers = calculate_centers(vert_dicts)
    
    for level in centers:
        centers_per_study["study_id"].append(row['study_id'])
        centers_per_study["series_id"].append(row['series_id'])
        centers_per_study["level"].append(level)
        
        centers_per_study["x"].append(centers[level][0])
        centers_per_study["y"].append(centers[level][1])
        centers_per_study["instance_number"].append(centers[level][2])


180it [04:27,  1.37s/it]

merging
merging vert 7 with 3


241it [05:54,  1.86s/it]

merging
merging vert 7 with 4


418it [10:21,  1.66s/it]

...

1808it [44:45,  1.41s/it]

merging
merging vert 5 with 1


1962it [48:34,  1.28s/it]

merging
merging vert 5 with 1


1980it [48:59,  1.48s/it]


In [11]:
centers_per_study = pd.DataFrame.from_dict(centers_per_study)
centers_per_study

,study_id,series_id,x,y,instance_number,level
0,4003253,1054713880,171.570871,116.577009,8,L1
1,4003253,1054713880,169.672123,157.646825,8,L2
2,4003253,1054713880,166.796317,202.517857,8,L3
3,4003253,1054713880,168.193304,246.345536,8,L4
4,4003253,1054713880,180.032143,285.492411,7,L5
...,...,...,...,...,...,...
11844,4290709089,4237840455,185.137175,125.306412,7,L2
11845,4290709089,4237840455,179.620908,171.211682,7,L3
11846,4290709089,4237840455,182.849026,216.144075,7,L4
11847,4290709089,4237840455,195.545860,259.996347,7,L5


In [12]:
centers_per_study.to_csv('../../data/SpineNet/centers_per_study.csv', index=False)

In [15]:
def convert_coords_to_patient(x, y, dicom_slice):
    
    # transform_matrix_factor = np.matrix(
    #     [[0, 1, 0, 0],
    #      [1, 0, 0, 0],
    #      [0, 0, 1, 0],
    #      [0, 0, 0, 1]]
    # )
        
    dX, dY = dicom_slice.PixelSpacing
    
    X = np.array(list(dicom_slice.ImageOrientationPatient[:3]) + [0]) * dY
    Y = np.array(list(dicom_slice.ImageOrientationPatient[3:]) + [0]) * dX

    S = np.array(list(dicom_slice.ImagePositionPatient) + [1])

    transform_matrix = np.array([Y, X, np.zeros(len(X)), S]).T
    # transform_matrix = transform_matrix @ transform_matrix_factor

    return (transform_matrix @ np.array([y, x, 0, 1]).T)

In [17]:
train_images_basepath = "../../data/rsna-2024-lumbar-spine-degenerative-classification/train_images"

patient_coords_dict = {
    "study_id": [],
    "level": [],
    "x": [],
    "y": [],
    "z": []
}

for index, group in centers_per_study.groupby("study_id"):
    for row_index, row in group.iterrows():
        dicom_slice_path = f"{train_images_basepath}/{row['study_id']}/{row['series_id']}/{row['instance_number']}.dcm"
        dicom_slice = dcmread(dicom_slice_path)
        coords = convert_coords_to_patient(row['x'], row['y'], dicom_slice)
        
        patient_coords_dict["study_id"].append(row['study_id'])
        patient_coords_dict["level"].append(row['level'])
        patient_coords_dict["x"].append(coords[0])
        patient_coords_dict["y"].append(coords[1])
        patient_coords_dict["z"].append(coords[2])
    
patient_coords = pd.DataFrame.from_dict(patient_coords_dict)
patient_coords

,study_id,level,x,y,z
0,4003253,L1,2.192236,57.393290,-373.897314
1,4003253,L2,2.417129,55.910159,-405.982332
2,4003253,L3,2.663579,53.663727,-441.036977
3,4003253,L4,2.900401,54.755404,-475.276522
4,4003253,L5,-1.697765,63.999043,-505.892668
...,...,...,...,...,...
11844,4290709089,L2,1.119082,47.663407,-359.280225
11845,4290709089,L3,0.473506,40.302511,-394.637869
11846,4290709089,L4,-0.206997,39.813034,-429.821794
11847,4290709089,L5,-0.924925,46.765941,-464.796865


In [18]:
patient_coords.to_csv('../../data/SpineNet/coords_3d.csv', index=False)

In [63]:
patient_bounding_boxes_dict = {
    "study_id": [],
    "level": [],
    "x_min": [],
    "y_min": [],
    "z_min": [],
    "x_max": [],
    "y_max": [],
    "z_max": [],
}

for index, group in patient_coords.groupby("study_id"):
    ordered_group = group.sort_values(by="level", ascending=True)
    if len(ordered_group) != 6:
        continue
    for level_index in range(5):
        patient_bounding_boxes_dict["study_id"].append(ordered_group['study_id'].iloc[0])
        level_label = ordered_group['level'].iloc[level_index].lower() + "_" + ordered_group['level'].iloc[level_index + 1] 
        patient_bounding_boxes_dict["level"].append(level_label)
        
        # Middle vertebra points
        pt_0 = np.array(ordered_group.iloc[level_index][["x", "y", "z"]])
        pt_1 = np.array(ordered_group.iloc[level_index + 1][["x", "y", "z"]])
        
        # Distance vector to the next vertebra
        d_vec = np.array(pt_0 - pt_1)
        d_size = np.linalg.norm(d_vec)
        d_unit = d_vec / d_size
        
        
        # Get a pair of orthogonal vectors to find x and y boundary candidates
        orth_1 = np.random.randn(3).astype(np.float64)
        orth_1 = orth_1 - orth_1.dot(d_unit) * d_unit
        orth_1 = orth_1 / np.linalg.norm(orth_1)
        
        orth_1 = orth_1.astype(np.float64)
        d_unit = d_unit.astype(np.float64)
        
        orth_2 = np.cross(orth_1, d_unit)
        orth_2 = orth_2.astype(np.float64)
        
        orth_1 *= d_size
        orth_2 *= d_size
        
        # Get candidate points (10 of them, 2 per orthogonal per each vertebra center, and the centers themselves)
        c_pts = np.array([pt - vec for pt in (pt_0, pt_1) for vec in (orth_1, orth_2)] + 
                         [pt + vec for pt in (pt_0, pt_1) for vec in (orth_1, orth_2)] +
                         [pt_0, pt_1])
        
        # x_min and x_max are just the min and max from all this
        x_min = np.min(c_pts[:, 0])
        x_max = np.max(c_pts[:, 0])
                
        # y_max is going to be over the center ys
        # And we're going to get y_min by getting y_min over c_pts and then extending the y_min over center ys
        c_pts_y_min = np.min(c_pts[:, 1])
        c_pts_y_max = np.max(c_pts[:, 1])

        y_max = max(pt_0[1], pt_1[1])
        y_min = min(pt_0[1], pt_1[1])
        
        y_max += abs(c_pts_y_max - y_max) * 2
        y_min -= abs(c_pts_y_min - y_min) / 2
        
        # z_max and z_min will be the same as x_min and x_max
        z_min = np.min(c_pts[:, 2])
        z_max = np.max(c_pts[:, 2])    
        
        patient_bounding_boxes_dict["x_min"].append(x_min)
        patient_bounding_boxes_dict["y_min"].append(y_min)
        patient_bounding_boxes_dict["z_min"].append(z_min)
        patient_bounding_boxes_dict["x_max"].append(x_max)
        patient_bounding_boxes_dict["y_max"].append(y_max)
        patient_bounding_boxes_dict["z_max"].append(z_max)

patient_bounding_boxes = pd.DataFrame.from_dict(patient_bounding_boxes_dict)
patient_bounding_boxes

,study_id,level,x_min,y_min,z_min,x_max,y_max,z_max
0,4003253,l1_L2,-29.829393,39.916436,-407.478494,34.438757,121.368184,-372.401152
1,4003253,l2_L3,-22.960396,41.006076,-442.830025,28.041104,106.540762,-404.189284
2,4003253,l3_L4,-27.854268,38.410474,-476.141536,33.418249,115.768414,-440.171964
3,4003253,l4_L5,-32.559214,40.005673,-516.058982,33.761850,122.997966,-465.110208
4,4003253,l5_S1,-28.139596,53.241165,-548.836007,24.912645,126.563647,-490.521556
...,...,...,...,...,...,...,...,...
9670,4290709089,l1_L2,-31.338473,32.218877,-369.266432,34.181214,120.206545,-315.070584
9671,4290709089,l2_L3,-27.894917,26.373099,-400.029531,29.487504,103.381058,-353.888563
9672,4290709089,l3_L4,-31.033451,24.400699,-430.578789,31.299960,101.951853,-393.880875
9673,4290709089,l4_L5,-31.902367,24.653177,-471.186880,30.770446,107.405369,-423.431779


In [64]:
patient_bounding_boxes.to_csv('../../data/SpineNet/bounding_boxes_3d.csv', index=False)